## MNIST Digit Recognizer CNN

#### Import TensorFlow into the program

In [6]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.8.0


### Load and Prepare the MNIST dataset

In [7]:
mnist = tf.keras.datasets.mnist                              # getting the MNIST dataset from keras

(x_train, y_train), (x_test, y_test) = mnist.load_data()     # doing the test and train split
x_train, x_test = x_train / 255.0, x_test / 255.0            # normalizing the pixel values

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")         # adding dimensions
x_test = x_test[..., tf.newaxis].astype("float32")

Usinf ```tf.data``` to batch and reshuffle the dataset

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Building the ```tf.keras``` model usnig Keras model subclassing API

In [9]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

#### Choosing an optimizer and loss function for training

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. Theese metrics accumulate the values over epochs and then print the overall result.

In [11]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use the ```tf.GradientTape``` to train the model

In [12]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

### Test the model

In [13]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [14]:
EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.13120925426483154, Accuracy: 96.00333404541016, Test Loss: 0.058929335325956345, Test Accuracy: 97.98999786376953
Epoch 2, Loss: 0.042195066809654236, Accuracy: 98.68999481201172, Test Loss: 0.06397895514965057, Test Accuracy: 98.13999938964844
Epoch 3, Loss: 0.02236928977072239, Accuracy: 99.24333190917969, Test Loss: 0.05580178648233414, Test Accuracy: 98.1500015258789
Epoch 4, Loss: 0.013433258980512619, Accuracy: 99.53666687011719, Test Loss: 0.05988061800599098, Test Accuracy: 98.30999755859375
Epoch 5, Loss: 0.008202935568988323, Accuracy: 99.76000213623047, Test Loss: 0.06544485688209534, Test Accuracy: 98.47000122070312
Epoch 6, Loss: 0.007855645380914211, Accuracy: 99.73500061035156, Test Loss: 0.0585893914103508, Test Accuracy: 98.55999755859375
Epoch 7, Loss: 0.005342915188521147, Accuracy: 99.81832885742188, Test Loss: 0.06396593153476715, Test Accuracy: 98.54000091552734
Epoch 8, Loss: 0.005949209909886122, Accuracy: 99.78666687011719, Test Loss: 0.0667285

The Image classifier is now trained to around 98-99% accuracy on this dataset.